In [1]:
!pip install git+https://github.com/openai/whisper.git
!apt-get install ffmpeg -y

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-mokkd6yq
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-mokkd6yq
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=74e1a16aaaaefe088d162d6561d0572072a95394c99eafb5336ef0a057440c3b
  Stored in directory: /tmp/pip-ephem-wheel-cache-zgc5_knf/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [12]:
from IPython.display import display, Javascript
from google.colab import output
import base64
import io
import IPython.display as ipd

# Global variable to store base64 audio
audio_data = None

def record_audio(seconds=5):
    display(Javascript(f"""
    async function record(seconds=5) {{
      const stream = await navigator.mediaDevices.getUserMedia({{audio:true}});
      const recorder = new MediaRecorder(stream);
      let chunks = [];
      recorder.ondataavailable = e => chunks.push(e.data);
      recorder.start();

      await new Promise(r => setTimeout(r, seconds * 1000));
      recorder.stop();

      await new Promise(r => recorder.onstop = r);
      const blob = new Blob(chunks, {{type:'audio/wav'}});
      const reader = new FileReader();

      reader.readAsDataURL(blob);
      reader.onloadend = () => {{
        const base64data = reader.result;
        google.colab.kernel.invokeFunction('notebook.saveAudio', [base64data], {{}});
      }};
    }}
    record({seconds});
    """))

def save_audio(data):
    global audio_data
    audio_data = data
    print("Audio recording saved in variable `audio_data`.")

output.register_callback('notebook.saveAudio', save_audio)

# Run this function to start recording and get microphone prompt
record_audio(seconds=5)

<IPython.core.display.Javascript object>

Audio recording saved in variable `audio_data`.


In [13]:
if audio_data:
    audio_bytes = base64.b64decode(audio_data.split(',')[1])
    with open('recorded_audio.wav', 'wb') as f:
        f.write(audio_bytes)
    print("Audio saved as recorded_audio.wav")
    ipd.display(ipd.Audio('recorded_audio.wav'))
else:
    print("No audio recorded yet. Run record_audio() first.")


Audio saved as recorded_audio.wav


In [14]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("recorded_audio.wav")
print(result["text"])

100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 43.1MiB/s]
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Hello my name is Pingley.
